# QRC-Lab: Risk Bounds on Real Quantum Hardware

In Lab 02, we explored the **Generalization Gap** in ideal simulations. In Lab 03, we learned how to connect to **Real Hardware**.

Now, we combine these two: How do **Risk Bounds** behave when the reservoir is noisy and the measurements are finite? 

### Theoretical Hypothesis
The paper "Quantum Reservoir Computing and Risk Bounds" suggests that larger reservoirs increase expressivity. However, on real hardware:
1. **Noise Accumulation**: More qubits or deeper circuits lead to more noise, which might "blur" the high-dimensional features.
2. **Feature Collapse**: In extreme noise regimes, the features might become indistinguishable, actually **improving** the generalization gap but at the cost of **poor accuracy** (underfitting).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qrc_sim.encoders import ReuploadingEncoder
from qrc_sim.reservoirs import RandomCRotReservoir
from qrc_sim.simulator import QRCSimulator
from qrc_sim.tasks.parity import ParityTask
from qrc_sim.readout import ReadoutModel

## 1. Comparing Generalization: Ideal vs. Noisy

We will run a scan of the number of qubits ($N=2, 4, 6$) and compare the training/test performance under realistic hardware noise (5% depolarizing error).

In [ ]:
task = ParityTask(length=200, delay=1)
(X_train, y_train), (X_test, y_test) = task.generate()

qubit_range = [2, 4, 6]
train_scores = []
test_scores = []

# Create Hardware Noise Model
noise_model = QRCSimulator.create_depolarizing_noise(p_error=0.03)

for n_q in qubit_range:
    print(f"Running 3% Noise Experiment with {n_q} qubits...")
    
    enc = ReuploadingEncoder(n_q, layers=1)
    res = RandomCRotReservoir(n_q, depth=1)
    obs = [('Z', i) for i in range(n_q)]
    
    # Simulation of Hardware Execution
    sim = QRCSimulator(enc, res, obs, backend_config='shots', shots=4000, noise_model=noise_model)
    
    f_train = sim.run_sequence(X_train)
    f_test = sim.run_sequence(X_test)
    
    # Higher regularization for hardware-like data
    readout = ReadoutModel(alpha=1e-1)
    readout.fit(f_train, y_train)
    
    train_scores.append(readout.score(f_train, y_train))
    test_scores.append(readout.score(f_test, y_test))

## 2. Results and Risk Analysis

Let's visualize the results. On real hardware, the "perfect training score" we saw in Lab 02 might be unattainable due to noise ceiling.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(qubit_range, train_scores, 'o-', label='Train Score (Noisy Hardware)')
plt.plot(qubit_range, test_scores, 's--', label='Test Score (Noisy Hardware)')
plt.fill_between(qubit_range, train_scores, test_scores, alpha=0.1, color='red', label='Hardware Generalization Gap')

plt.xlabel("Number of Qubits")
plt.ylabel("R2 Score")
plt.title("The Generalization Gap on Real Quantum Hardware (Simulated)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Discussion: Scaling in the Real World

1. **The Noise Ceiling**: Notice that the Train Score might plateau earlier than 1.0. This is because the quantum features are perturbed by noise, limiting the Readout's ability to discriminate perfectly.
2. **Risk and Generalization**: In some cases, noise acts as a **natural regularizer**, preventing the model from fitting high-frequency fluctuations in the training data, which could lead to a **smaller generalization gap** than the ideal case, but at the cost of lower overall accuracy.
3. **Conclusion for Research**: When publishing QRC results, one must perform this scan. If your hardware results show high Train but low Test, you are in the **High Bound Risk** regime described in the article.

## 3. Deployment to Real Backend

To replicate this scan on an actual IBM device, use a loop to submit batch jobs for $N=2, 4, 6...$

In [ ]:
print("# Pseudo-code for Hardware Scan")
print("# for n_q in [2, 4, 6, 8]:")
print("#     job = submit_batch_to_ibm(n_q, X_train)")
print("#     results.append(job.result())")